In [9]:
import pandas as pd
import sqlite3


In [10]:
# Conectar (crear en este caso) con la base de datos.

connection = sqlite3.connect("ECI.db")

# Crear cursor para realizar consultas.
cursor = connection.cursor()


def sql_query(query, con):
    '''
    Función para leer datos y pasarlos a pandas DF.
    '''
    cursor = con.cursor()
    cursor.execute(query)
    # Almacena los datos de la query 
    ans = cursor.fetchall()
    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in cursor.description]
    
    return pd.DataFrame(ans,columns=names)

In [11]:
# Creación de pandas DFs

dict_DEPTO_CAMPANIA = {'ID_DPTO':['1','1','2', '3', '3', '3', '4', '4', '4', '4'],
                'ID_CAMPANIA':['CA1', 'CA4', 'CA2', 'CA2', 'CA5', 'CA6', 'CA1', 'CA2', 'CA3', 'CA6']}
DEPTO_CAMPANIA = pd.DataFrame(dict_DEPTO_CAMPANIA)

DATA_DEPARTAMENTOS = {
        'ID_DPTO': ['1', '2', '3', '4'],
        'DESC_DPTO': ['JOYERIA', 'BOLSOS', 'ZAPATERIA', 'BAÑO CAB.']}
DEPARTAMENTO = pd.DataFrame(DATA_DEPARTAMENTOS)

familia = [['1', '1', '2', '3', '3', '4'], 
           ['1F1', '1F2', '2F1', '3F1', '3F2', '4F1'], 
           ['PULSERAS', 'ANILLOS', 'BOLSOS DE MANO', 'NAUTICO', 'SANDALIAS', 'BERMUDAS']]

columnas = ['ID_DPTO', 'COD_FAMILIA', 'DESC_FAMILIA']
FAMILIA = pd.DataFrame(dict(zip(columnas, familia)))

dic_campanias =  { "ID_CAMPANIA" : ["CA1", "CA2", "CA3", "CA4", "CA5", "CA6"], "TIPO": ["VENTA", "VENTA", "POSVENTA", "VENTA", "POSVENTA", "POSVENTA"], "FECHA_CAMP" : ["05/02/2012", "04/02/2012", "02/10/2009", "06/05/2010", "30/11/2010", "07/02/2015"] }
CAMPANIA = pd.DataFrame(dic_campanias)

Referencias = pd.Series(['A','B','C','D','E','F','G','H'])
ID_DPTO = pd.Series(['1','2','2','2','2','3','3','4'])
COD_FAMILIA =pd.Series(['1F1','2F1','2F1','2F1','2F1','3F1','3F2','4F1'])
FECHA_ALTA = pd.Series(['21/02/2010','07/02/2014','15/05/2014','01/10/2005','31/10/2010','20/04/2001','14/02/2015','25/03/2009'])
ARTICULO = pd.DataFrame({'REFERENCIA': Referencias,'ID_DPTO': ID_DPTO,'COD_FAMILIA':COD_FAMILIA,'FECHA_ALTA':FECHA_ALTA})

talon = ['01', '02', '03', '04', '05', '06', '07', '08']
referencia = ['A', 'A', 'A', 'D', 'E', 'E', 'H', 'H']
precio = [15, 18, 14, 37.95, 125.90, 150, 22.99, 24]
fecha_venta = ['11/02/2012', '15/02/2012', '02/10/2013', '06/08/2010', '30/11/2010', '05/02/2011', '11/04/2011', '08/08/2011']
VENTA = pd.DataFrame({'TALON':talon,
                    'REFERENCIA':referencia,
                    'PRECIO':precio,
                    'FECHA_VENTA':fecha_venta})


In [12]:
# Incorporación de los DFs como tablas en la base de datos
DEPTO_CAMPANIA.to_sql("DEPTO_CAMPANIA", connection, if_exists='replace',index=False)
DEPARTAMENTO.to_sql("DEPARTAMENTO", connection, if_exists='replace',index=False)
DEPARTAMENTO.to_sql("DEPARTAMENTO", connection, if_exists='replace',index=False)
FAMILIA.to_sql("FAMILIA", connection, if_exists='replace',index=False)
CAMPANIA.to_sql("CAMPANIA", connection, if_exists='replace',index=False)
ID_DPTO.to_sql("ID_DPTO", connection, if_exists='replace',index=False)
COD_FAMILIA.to_sql("COD_FAMILIA", connection, if_exists='replace',index=False)
FECHA_ALTA.to_sql("FECHA_ALTA", connection, if_exists='replace',index=False)
ARTICULO.to_sql("ARTICULO", connection, if_exists='replace',index=False)
VENTA.to_sql("VENTA", connection, if_exists='replace',index=False)


In [13]:
# Para hacer las consultas en SQL se usará esta función:
# sql_query("comando en sql", connection)

In [102]:
sql_query("SELECT * FROM VENTA", connection)

,TALON,REFERENCIA,PRECIO,FECHA_VENTA
0,01,A,15.00,11/02/2012
1,02,A,18.00,15/02/2012
2,03,A,14.00,02/10/2013
3,04,D,37.95,06/08/2010
4,05,E,125.90,30/11/2010
5,06,E,150.00,05/02/2011
6,07,H,22.99,11/04/2011
7,08,H,24.00,08/08/2011


## 2. Genera una consulta que obtenga la lista ordenada de todas las referencias sin venta

In [23]:
sql_query("SELECT * FROM ARTICULO LEFT JOIN VENTA ON VENTA.REFERENCIA = ARTICULO.REFERENCIA WHERE VENTA.TALON IS NULL", connection)

,REFERENCIA,ID_DPTO,COD_FAMILIA,FECHA_ALTA,TALON,REFERENCIA,PRECIO,FECHA_VENTA
0,B,2,2F1,07/02/2014,None,None,None,None
1,C,2,2F1,15/05/2014,None,None,None,None
2,F,3,3F1,20/04/2001,None,None,None,None
3,G,3,3F2,14/02/2015,None,None,None,None


## 3. Genera una consulta que obtenga las ventas comprendidas entre 2010 y 2011 (ambos incluidos), que estén asociados a una campaña de tipo VENTA y que sean del departamento de JOYERÍA.

In [197]:
# Este era tricky, tenía fallo y no salía nada.

sql_query('''SELECT * FROM VENTA
           INNER JOIN ARTICULO
            ON VENTA.REFERENCIA = ARTICULO.REFERENCIA
            INNER JOIN DEPTO_CAMPANIA
            ON ARTICULO.ID_DPTO = DEPTO_CAMPANIA.ID_DPTO
            INNER JOIN CAMPANIA
            ON DEPTO_CAMPANIA.ID_CAMPANIA = CAMPANIA.ID_CAMPANIA
            WHERE "TIPO" = "VENTA" AND
            DEPTO_CAMPANIA.ID_DPTO = 1 AND
            strftime('%Y', FECHA_VENTA) BETWEEN '2010' AND '2011' ''', connection)

,TALON,REFERENCIA,PRECIO,FECHA_VENTA,REFERENCIA,ID_DPTO,COD_FAMILIA,FECHA_ALTA,ID_DPTO,ID_CAMPANIA,ID_CAMPANIA,TIPO,FECHA_CAMP


## 4. Genere una consulta que cree un campo “co_ranking” que clasifique en orden ascendente las campañas según el total de venta, siendo 1 la campaña que más ha vendido y N la que menos. (N = Total de campañas)

In [97]:
sql_query('''SELECT CAMPANIA.ID_CAMPANIA,
                    SUM(VENTA.PRECIO) AS VENTA_TOTAL,
                    ROW_NUMBER() OVER(ORDER BY SUM(VENTA.PRECIO) DESC) AS RANKING

          FROM CAMPANIA
          
          LEFT JOIN DEPTO_CAMPANIA ON CAMPANIA.ID_CAMPANIA = DEPTO_CAMPANIA.ID_CAMPANIA
          LEFT JOIN DEPARTAMENTO ON DEPARTAMENTO.ID_DPTO = DEPTO_CAMPANIA.ID_DPTO
          LEFT JOIN ARTICULO ON ARTICULO.ID_DPTO = DEPARTAMENTO.ID_DPTO
          LEFT JOIN VENTA ON ARTICULO.REFERENCIA = VENTA.REFERENCIA

          GROUP BY CAMPANIA.ID_CAMPANIA         
          ''', connection)

,ID_CAMPANIA,VENTA_TOTAL,RANKING
0,CA2,360.84,1
1,CA1,93.99,2
2,CA4,47.00,3
3,CA3,46.99,4
4,CA6,46.99,5
5,CA5,NaN,6


## 5. Clasifique todas las ventas en Mayor, Igual o Menor precio respecto a la media de los precios de todas las ventas realizadas.

In [201]:
sql_query(""" SELECT *,
       CASE
           WHEN PRECIO > (SELECT AVG(PRECIO) FROM VENTA) THEN 'Mayor'
           WHEN PRECIO = (SELECT AVG(PRECIO) FROM VENTA) THEN 'Igual'
           ELSE 'Menor'
       END AS clasificacion_precio
FROM VENTA""", connection)

,TALON,REFERENCIA,PRECIO,FECHA_VENTA,clasificacion_precio
0,01,A,15.00,11/02/2012,Menor
1,02,A,18.00,15/02/2012,Menor
2,03,A,14.00,02/10/2013,Menor
3,04,D,37.95,06/08/2010,Menor
4,05,E,125.90,30/11/2010,Mayor
5,06,E,150.00,05/02/2011,Mayor
6,07,H,22.99,11/04/2011,Menor
7,08,H,24.00,08/08/2011,Menor


# 7. Genere una consulta que para cada campaña de tipo “venta” obtenga el talón medio, precio medio, nº de referencias compradas y nº de talones con venta superior al talón medio.

In [198]:
sql_query('''SELECT CAMPANIA.ID_CAMPANIA,
        CAMPANIA.TIPO,
        AVG(VENTA.TALON) AS talon_medio,
        AVG(VENTA.PRECIO) AS precio_medio,
        COUNT(ARTICULO.REFERENCIA) AS num_referencias,
        SUM(CASE WHEN VENTA.TALON > (SELECT AVG(TALON) FROM VENTA) THEN 1 ELSE 0 END) AS num_talones_ventas_superior

        FROM CAMPANIA

        INNER JOIN DEPTO_CAMPANIA
        ON CAMPANIA.ID_CAMPANIA = DEPTO_CAMPANIA.ID_CAMPANIA
        INNER JOIN ARTICULO
        ON DEPTO_CAMPANIA.ID_DPTO = ARTICULO.ID_DPTO
        INNER JOIN VENTA
        ON ARTICULO.REFERENCIA = VENTA.REFERENCIA

        WHERE CAMPANIA.TIPO = 'VENTA'
        
        GROUP BY CAMPANIA.ID_CAMPANIA, CAMPANIA.TIPO;
             ''', connection)

,ID_CAMPANIA,TIPO,talon_medio,precio_medio,num_referencias,num_talones_ventas_superior
0,CA1,VENTA,4.2,18.798000,5,0
1,CA2,VENTA,6.0,72.168000,5,0
2,CA4,VENTA,2.0,15.666667,3,0


## 8. Genere una o varias consultas que devulevan las combinaciones de familias y campañas cuya venta haya sido superior a 100€.

In [199]:
sql_query('''SELECT CAMPANIA.ID_CAMPANIA, CAMPANIA.TIPO, FAMILIA.DESC_FAMILIA,
             SUM(VENTA.PRECIO) as VENTA_TOTAL

             FROM CAMPANIA

             INNER JOIN DEPTO_CAMPANIA ON CAMPANIA.ID_CAMPANIA = DEPTO_CAMPANIA.ID_CAMPANIA
             INNER JOIN FAMILIA ON DEPTO_CAMPANIA.ID_DPTO = FAMILIA.ID_DPTO
             INNER JOIN ARTICULO ON FAMILIA.ID_DPTO = ARTICULO.ID_DPTO
             INNER JOIN VENTA ON ARTICULO.REFERENCIA = VENTA.REFERENCIA

             GROUP BY CAMPANIA.ID_CAMPANIA, FAMILIA.DESC_FAMILIA
             
             HAVING SUM(VENTA.PRECIO) > 100''', connection)

,ID_CAMPANIA,TIPO,DESC_FAMILIA,VENTA_TOTAL
0,CA2,VENTA,BOLSOS DE MANO,313.85


## 9. Genere una o varias consultas que permitan catalogar los artículos vendidos y que no son de la Campaña CA1, según si se tratan de artículos de Ropa o Accesorios. En base a los departamentos y familias. NOTA: Se valorará la elaboración de la consulta y no la veracidad del propio catálogo)

In [196]:
sql_query('''SELECT DISTINCT VENTA.REFERENCIA, DESC_DPTO,

             CASE
                WHEN FAMILIA.ID_DPTO = 1 THEN "ACCESORIO" ELSE "ROPA"
             END AS TIPO

             FROM VENTA
             LEFT JOIN ARTICULO ON VENTA.REFERENCIA = ARTICULO.REFERENCIA
             LEFT JOIN DEPTO_CAMPANIA ON ARTICULO.ID_DPTO = DEPTO_CAMPANIA.ID_DPTO
             LEFT JOIN DEPARTAMENTO ON DEPTO_CAMPANIA.ID_DPTO = DEPARTAMENTO.ID_DPTO
             LEFT JOIN FAMILIA ON DEPARTAMENTO.ID_DPTO = FAMILIA.ID_DPTO
             LEFT JOIN CAMPANIA ON DEPTO_CAMPANIA.ID_CAMPANIA = CAMPANIA.ID_CAMPANIA

             WHERE CAMPANIA.ID_CAMPANIA != "CA1"
             ''', connection)


,REFERENCIA,DESC_DPTO,TIPO
0,A,JOYERIA,ACCESORIO
1,D,BOLSOS,ROPA
2,E,BOLSOS,ROPA
3,H,BAÑO CAB.,ROPA


## 10. Genere una o varias consultas que devuelvan la variación del total de venta entre años. Tomando como variación la siguiente fórmula $$\dfrac{Year_{n} – Year_{n-1}}{Year_{n-1}}$$ (Se valorará el control de valores 0 en la división o de indeterminaciones)

In [200]:
sql_query('''SELECT anio AS año_1,
       LAG(anio) OVER (ORDER BY anio DESC) AS año_2,
       (LAG(total_anio) OVER (ORDER BY anio DESC) - total_anio) /total_anio  AS variacion

        FROM (SELECT SUBSTR(fecha_venta, -4) AS anio, SUM(precio) as total_anio
FROM venta

GROUP BY anio
ORDER BY anio DESC)
ORDER BY anio DESC''', connection)

,año_1,año_2,variacion
0,2013,None,NaN
1,2012,2013,-0.575758
2,2011,2012,-0.832479
3,2010,2011,0.202258
